## Merge area lists from Propsearch.com into a single list

Having scrapped area lists containing all projects/buildings in each from propsearch which have been saved in separate files and lists, we seek to merge all into a single list to classify by completion/development/hold/planned. This will help us inform which developers to contact for new and ongoing off-plan sales.

This list will contain the following columns:
* Link to the particular building/community on propsearch.com: link to Marina Gate 2
* Building/project name: e.g. Marina Gate 2
* Area: e.d. Dubai Marina
* Completion stage: e.g. completed

The list is saved as a csv file for further use.

In [241]:
import pandas as pd
import numpy as np
import seaborn as sns

In [187]:
import glob
import os

def preprocess_single_projects(path):
    '''
    Merge all csv files in the directory into single file
    
    Input: directory: directory name, not path, assumes this notebook is in the 
    '''
    # read in all file paths
    # assumes they are in the cwd
    path_list = glob.glob('./{}/*/*'.format(directory))
    
    # process paths into df
    projects_df = pd.DataFrame()
    for path in path_list:
        raw_df = pd.read_csv(os.path.abspath(path), index_col=False)
        if 'ps-loc-card-location' not in raw_df.columns:
            # extract location name from path
            added_location = path.split('/')[2].replace('-', ' ').title()
            # add area name to new column
            raw_df.insert(loc=0, column='ps-loc-card-location', value=added_location)
        # add custom area name column based on the name I used for each directory containing an area
        added_location = path.split('/')[2].replace('-', ' ').title() #todo
        raw_df.insert(loc=0, column='area', value=added_location) #todo
        projects_df = pd.concat([projects_df, raw_df], axis=0)
    
    # clean df
    #drop 'lazy src column'
    projects_df.drop(columns=['lazy src'], inplace=True)
    # reorder columns
    new_column_order = ['area', 'ps-loc-card-location', 'ps-loc-card-title', 
                        'ps-loc-card-status', 'tablescraper-selected-row href']
    projects_df = projects_df[new_column_order]
    # rename columns to simpler names
    renamed_columns = {'ps-loc-card-location': 'sub_area', 'ps-loc-card-title': 'project', 
                        'ps-loc-card-status': 'status', 'tablescraper-selected-row href': 'link'}
    projects_df.rename(columns=renamed_columns, inplace=True)
    # sort by area name
    projects_df.sort_values(by=['area'], inplace=True)
    # reset indices
    projects_df.reset_index(drop=True, inplace=True)
    
    return projects_df

In [188]:
# read and preprocess data into df containing all towers
directory = 'buildings_raw'
towers_df = preprocess_single_projects(directory)

In [189]:
# describe data
towers_df.describe()

,area,sub_area,project,status,link
count,1576,1576,1576,1576,1576
unique,13,57,1576,11,1576
top,Jumeirah Village Circle,Business Bay,Lana Tower,Complete,https://propsearch.ae/dubai/golden-wood-view
freq,498,201,1,934,1


#### Save towers_df to csv

In [1072]:
towers_df.to_csv('towers_df.csv', index=False)

#### Look at data an extract desired projects to look into

In [194]:
towers_df.head(3)

,area,sub_area,project,status,link
0,Arjan,Arjan,Dania Building 2,Complete,https://propsearch.ae/dubai/dania-building-2
1,Arjan,Arjan,Joya Blanca,Under development (In progress),https://propsearch.ae/dubai/joya-blanca
2,Arjan,Arjan,La Fontana,Complete,https://propsearch.ae/dubai/la-fontana


In [191]:
# project count by status type
towers_df.status.value_counts()

Complete                             934
Under development                    277
Under development (Cancelled)        136
Under development (On hold)          134
Planned                               47
Envisioned                            22
Under development (In progress)       15
Planned (Cancelled)                    5
Under development (Progress slow)      3
Demolished                             2
Complete (Handover underway)           1
Name: status, dtype: int64

In [192]:
# projects under development by area
on_dev_df = towers_df[towers_df.status == 'Under development']
on_dev_df.area.value_counts()

Jumeirah Village Circle      97
Business Bay                 34
Arjan                        32
Jumeirah Village Triangle    22
Downtown                     18
Dubai Creek Harbour          17
Palm Jumeirah                16
Sobha Hartland               11
Dubai Hills Estate           10
Dubai Marina                  9
Sports City                   7
Jumeirah Lakes Towers         2
Motor City                    2
Name: area, dtype: int64

In [548]:
# projects under development in Business Bay
bbay_on_dev_df = on_dev_df[on_dev_df.area == 'Business Bay']
bbay_on_dev_df

,area,sub_area,project,status,link
96,Business Bay,Business Bay,Millennium Binghatti Residences,Under development,https://propsearch.ae/dubai/millennium-binghat...
98,Business Bay,Business Bay,Moon Tower,Under development,https://propsearch.ae/dubai/moon-tower
99,Business Bay,Business Bay,Mövenpick Hotel & Living,Under development,https://propsearch.ae/dubai/movenpick-hotel-li...
108,Business Bay,Business Bay,Nobles Residential Tower,Under development,https://propsearch.ae/dubai/nobles-residential...
110,Business Bay,Business Bay,Marble Arch Tower,Under development,https://propsearch.ae/dubai/marble-arch-tower
118,Business Bay,Business Bay,Mama Shelter Dubai,Under development,https://propsearch.ae/dubai/mama-shelter-dubai
121,Business Bay,Business Bay,Lillian Tower,Under development,https://propsearch.ae/dubai/lillian-tower
133,Business Bay,Business Bay,AG Tower Business Bay,Under development,https://propsearch.ae/dubai/ag-tower-business-bay
134,Business Bay,Business Bay,Ahad Residences,Under development,https://propsearch.ae/dubai/ahad-residences
140,Business Bay,Business Bay,Paramount Tower Hotel & Residences,Under development,https://propsearch.ae/dubai/paramount-tower-ho...


In [199]:
bbay_on_dev_df.link.iloc[0]

'https://propsearch.ae/dubai/millennium-binghatti-residences'

In [178]:
towers_df.sub_area.unique()

array(['Jumeirah Lakes Towers', 'Sports City', 'Canal Residence West',
       'Sobha Hartland', 'Sobha Hartland Greens', 'JVC District 15',
       'JVC District 14', 'JVC District 12', 'JVC District 18',
       'JVC District 11', 'JVC District 17', 'JVC District 10',
       'JVC District 13', 'JVC District 16', 'Jumeirah Village Circle',
       'Downtown Dubai', 'The Opera District', 'Yansoon', 'Zaafaran',
       'Zanzabeel', 'Old Town', 'Kamoon', 'Miska', 'Reehan',
       'Al Murooj Complex', 'Dubai Marina', 'Jumeirah Beach Residence',
       'Bluewaters Island', 'Dubai Marina Mall', 'Bluewaters Residences',
       'The Address Dubai Marina', 'Creek Island', 'Creek Beach',
       'Dubai Creek Harbour', 'Uptown Motor City', 'Motor City',
       'Green Community Motor City', 'Business Bay', 'Al Habtoor City',
       'Bay Square', 'M Hotel Downtown by Millennium Dubai', 'Arjan',
       'Palm Jumeirah', 'The Palm Crescent', 'The Golden Mile',
       'Park Heights', 'Dubai Hills Estate', '

In [179]:
towers_df.area.unique()

array(['Jumeirah Lakes Towers', 'Sports City', 'Sobha Hartland',
       'Jumeirah Village Circle', 'Downtown', 'Dubai Marina',
       'Dubai Creek Harbour', 'Motor City', 'Business Bay', 'Arjan',
       'Palm Jumeirah', 'Dubai Hills Estate', 'Jumeirah Village Triangle'],
      dtype=object)

## Get gps coordinates and project tabled data for each project

Using Beatutifulsoup to scrap each link on the project df
* key information
* companies-associated
* milestones
* proximity-to-landmarks
* coordinates

Scrapping inspiration from Datacamp [link](https://www.datacamp.com/community/tutorials/amazon-web-scraping-using-beautifulsoup?utm_source=adwords_ppc&utm_campaignid=898687156&utm_adgroupid=48947256715&utm_device=c&utm_keyword=&utm_matchtype=b&utm_network=g&utm_adpostion=&utm_creative=229765585183&utm_targetid=dsa-429603003980&utm_loc_interest_ms=&utm_loc_physical_ms=1000013&gclid=Cj0KCQjw0emHBhC1ARIsAL1QGNdx-wU43XjIPm2mfMWut-ceTSl2j3WZ7X8DiH4tl7mfG4njEnGfGYkaAuCiEALw_wcB) and Real Python [link](https://realpython.com/beautiful-soup-web-scraper-python/)

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import re

In [550]:
len(bbay_on_dev_df)

34

In [633]:
import re

def get_last_photo_update(soup):
    # get string and link of last project photo update to check progress
    construction_updates = soup.find(id='jump-to-construction-updates')
    gallery = construction_updates.find('div', class_='fotorama ps-gallery-portrait')
    raw_caption = gallery.find('a')

    raw_caption = str(raw_caption)
    caption = re.search('data-caption="(.*) Image &amp;copy', raw_caption)
    caption = caption.group(1).replace('\"', '')
    photo_url = re.search('data-full=(.*) href=', raw_caption)
    photo_url = photo_url.group(1).replace('\"', '')
    return caption, photo_url

In [636]:
# alias for the dataframe containing the links to scrap
scrap_df = bbay_on_dev_df
# set bounds for urls to scrap
url_idx_start = 0
url_idx_end = len(scrap_df) + 1
# construct an iterable with the original index and the url
url_list = zip(list(scrap_df.link.iloc[url_idx_start:url_idx_end].index), 
               list(scrap_df.link.iloc[url_idx_start:url_idx_end]))
# instantiate dic to hold scrapped data
scrapped_dict = {}
# iterate over a list of urls
for url_key, url in url_list:
    # cast url_key as str to index the dictionary
    url_key = str(url_key)
    # set request header with appropriate permissions
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    # request url data
    r = requests.get(url, headers=headers)#, proxies=proxies)
    # get content
    content = r.content
    # parse content
    soup = BeautifulSoup(content, "html.parser")
    
    # add this entry to dictionary, using the same key as in the original dataframe index
    scrapped_dict[url_key] = {}
    # save the url
    scrapped_dict[url_key]['url'] = url
    # add space to save each table with info
    scrapped_dict[url_key]['tables'] = {}

    # get coordinates
    coordinates_div = soup.find('div', attrs={'class':'ps-guide-sub-subheading'})
    coordinates = coordinates_div.text.strip().replace('place', '').replace('\'', ' ')
    # add coordinates to dict
    scrapped_dict[url_key]['coordinates'] = coordinates
    
    # get las photo update
    try:
        caption, photo_url = get_last_photo_update(soup)
        scrapped_dict[url_key]['last_photo_update'] = {'caption': caption, 'photo_url': photo_url}
    except:
        print('Failed to get photo update for: {}'.format(url))
        pass

    # get tables with contents
    kv_list = soup.findAll('div', class_='ps-kv-list')
    # iterate through tables
    for k in kv_list:
        # get table title
        # some tables won't have a title, which we ignore
        try:
            title = k.find(class_='ps-kv-list-title').text.lower().replace(' ', '-')
        except AttributeError as a_err:
            print('Error on table for: {}'.format(url))
            continue
        # instantiate dict for each title
        scrapped_dict[url_key]['tables'][title] = {}
        # get table content
        contents = k.findAll('div', class_='grid-x')
        # iterate through table content
        for content in contents:
            entry = content.text.strip().split('\n')
            scrapped_dict[url_key]['tables'][title][entry[0]] = entry[1]
            

Error on table for: https://propsearch.ae/dubai/mama-shelter-dubai
Error on table for: https://propsearch.ae/dubai/amna-tower
Error on table for: https://propsearch.ae/dubai/dorchester-hotel-dubai
Error on table for: https://propsearch.ae/dubai/zada-tower


In [637]:
scrapped_dict

{'96': {'url': 'https://propsearch.ae/dubai/millennium-binghatti-residences',
  'tables': {'millennium-binghatti-residences-key-information': {'Location type': 'Residential building',
    'Area': 'Business Bay',
    'Phase': 'Under development',
    'Floors': '24',
    'Building type': 'High-rise building',
    'Total units': '230',
    'Unit types': '113 studios58 one-bedroom apartments59 two-bedroom apartments'},
   'companies-associated-with-millennium-binghatti-residences': {'Developer': 'Binghatti Developers',
    'Architectural Consultant': 'Eng. Adnan Saffarini',
    'Contractor': 'Granada Europe Engineering Contracting Co.'},
   'millennium-binghatti-residences-milestones': {'Date Launched': 'March 2018',
    'Estimated Handover Date': 'Q4 2019',
    'Construction Started': '2018'},
   'proximity-to-landmarks': {'Palm Jumeirah': '20 mins drive',
    'Al Maktoum International Airport': '42 mins drive',
    'La Mer by Meraas': '19 mins drive',
    'Mall of the Emirates': '16 mins

## Save scrapped data (Business Bay) as json file

Online json tutorial [link](https://www.freecodecamp.org/news/python-read-json-file-how-to-load-json-from-a-file-and-parse-dumps/)

In [647]:
import json

# save file
json_dump_name = 'business-bay-under-development'
data_to_dump = scrapped_dict
with open(json_dump_name + '.json', 'w') as file:
    json.dump(data_to_dump, file)

In [921]:
# load json file into dictionary
json_load_file_name = 'business-bay-under-development'
with open(json_load_file_name + '.json') as file:
    # Load its content and make a new dictionary
    loaded_json_data = json.load(file)

In [922]:
loaded_json_data

{'96': {'url': 'https://propsearch.ae/dubai/millennium-binghatti-residences',
  'tables': {'millennium-binghatti-residences-key-information': {'Location type': 'Residential building',
    'Area': 'Business Bay',
    'Phase': 'Under development',
    'Floors': '24',
    'Building type': 'High-rise building',
    'Total units': '230',
    'Unit types': '113 studios58 one-bedroom apartments59 two-bedroom apartments'},
   'companies-associated-with-millennium-binghatti-residences': {'Developer': 'Binghatti Developers',
    'Architectural Consultant': 'Eng. Adnan Saffarini',
    'Contractor': 'Granada Europe Engineering Contracting Co.'},
   'millennium-binghatti-residences-milestones': {'Date Launched': 'March 2018',
    'Estimated Handover Date': 'Q4 2019',
    'Construction Started': '2018'},
   'proximity-to-landmarks': {'Palm Jumeirah': '20 mins drive',
    'Al Maktoum International Airport': '42 mins drive',
    'La Mer by Meraas': '19 mins drive',
    'Mall of the Emirates': '16 mins

### Build dataframe from scrapped data of projects

In [923]:
data_info_df = pd.DataFrame(loaded_json_data).T

In [924]:
data_info_df.head(2)

,url,tables,coordinates,last_photo_update
96,https://propsearch.ae/dubai/millennium-binghat...,{'millennium-binghatti-residences-key-informat...,"25°10 51""N 55°16 8.8""E","{'caption': 'Millennium Binghatti Residences, ..."
98,https://propsearch.ae/dubai/moon-tower,{'moon-tower-key-information': {'Location type...,"25°11 3.5""N 55°17 5.1""E","{'caption': 'Moon Tower, construction update M..."


In [925]:
# deserialized dataframe will have an 'object' type index, whereas the local copy is 'int64'
# reast deseralized index to match local, otherwise the concatenation operation returns nan values
data_info_df.index = data_info_df.index.astype('int64')
# concatenate dataframes
# bbay_on_dev_df: business bay under development
complete_df = pd.concat([bbay_on_dev_df, data_info_df], axis=1)
# drop duplicate url-link column
complete_df.drop(columns=['link'], inplace=True)

In [926]:
complete_df.head(2)

,area,sub_area,project,status,url,tables,coordinates,last_photo_update
96,Business Bay,Business Bay,Millennium Binghatti Residences,Under development,https://propsearch.ae/dubai/millennium-binghat...,{'millennium-binghatti-residences-key-informat...,"25°10 51""N 55°16 8.8""E","{'caption': 'Millennium Binghatti Residences, ..."
98,Business Bay,Business Bay,Moon Tower,Under development,https://propsearch.ae/dubai/moon-tower,{'moon-tower-key-information': {'Location type...,"25°11 3.5""N 55°17 5.1""E","{'caption': 'Moon Tower, construction update M..."


### Save full dataframe into csv

In [797]:
complete_df.to_csv('business-bay-under-development-merge.csv')

### Load merged csv to dataframe

CAVEAT: When loaded, data previously saved as a dictionary is cast as a string, which causes problems when trying to rename keys.  For this, either rename keys by using a dataframe built from loading the json file, not the loaded csv, or recast the necessary columns into dictionaries if you are using the loaded csv file.

In [798]:
complete_df = pd.read_csv('business-bay-under-development-merge.csv')

In [799]:
complete_df.head(2)

,Unnamed: 0,area,sub_area,project,status,url,tables,coordinates,last_photo_update
0,96,Business Bay,Business Bay,Millennium Binghatti Residences,Under development,https://propsearch.ae/dubai/millennium-binghat...,{'millennium-binghatti-residences-key-informat...,"25°10 51""N 55°16 8.8""E","{'caption': 'Millennium Binghatti Residences, ..."
1,98,Business Bay,Business Bay,Moon Tower,Under development,https://propsearch.ae/dubai/moon-tower,{'moon-tower-key-information': {'Location type...,"25°11 3.5""N 55°17 5.1""E","{'caption': 'Moon Tower, construction update M..."


## Extract key pieces of data from the tables column

This will surface data and place it in columns for easy referencing
* Total Units
* Developer name
* Date Launched
* Construction Started
* Estimated Handover Date

#### Rename dictionary keys in tables column for easy referencing

In [927]:
# function that renames the keys in the tables to remove the project name
def rename_table_keys(table):
    # Input: table: contents of each rown on the tables column of the dataframe
    # eg transform: 'mama-shelter-dubai-key-information' -->  #'key-information'
    # extract all available keys
    key_ids = list(table)
    # iterate over keys and rename them to remove the project name from each and make them universal keys
    for key_id in key_ids:
        if 'key-information' in key_id:
            table['key-information'] = table.pop(key_id)
        elif 'companies-associated' in key_id:
            table['companies-associated'] = table.pop(key_id)
        elif  'milestones' in key_id:
            table['milestones'] = table.pop(key_id)
    return table

In [928]:
# make a deep copy of the loaded dataframe to make changes to it without modifying original
renamed_complete_df = complete_df.copy(deep=True)

In [929]:
# apply rename_table_keys function to rename keys in tables column for easy referencing
# this removes the project name from the key strings in each dictionary
renamed_complete_df.tables.apply(rename_table_keys);

In [930]:
renamed_complete_df.head(5)

,area,sub_area,project,status,url,tables,coordinates,last_photo_update
96,Business Bay,Business Bay,Millennium Binghatti Residences,Under development,https://propsearch.ae/dubai/millennium-binghat...,{'proximity-to-landmarks': {'Palm Jumeirah': '...,"25°10 51""N 55°16 8.8""E","{'caption': 'Millennium Binghatti Residences, ..."
98,Business Bay,Business Bay,Moon Tower,Under development,https://propsearch.ae/dubai/moon-tower,{'proximity-to-landmarks': {'Palm Jumeirah': '...,"25°11 3.5""N 55°17 5.1""E","{'caption': 'Moon Tower, construction update M..."
99,Business Bay,Business Bay,Mövenpick Hotel & Living,Under development,https://propsearch.ae/dubai/movenpick-hotel-li...,{'proximity-to-landmarks': {'Palm Jumeirah': '...,"25°11 19.6""N 55°16 13.1""E",{'caption': 'Mövenpick Hotel Apartments Busine...
108,Business Bay,Business Bay,Nobles Residential Tower,Under development,https://propsearch.ae/dubai/nobles-residential...,{'proximity-to-landmarks': {'Palm Jumeirah': '...,"25°11 4.5""N 55°17 24.8""E","{'caption': 'Nobles Residential Tower, constru..."
110,Business Bay,Business Bay,Marble Arch Tower,Under development,https://propsearch.ae/dubai/marble-arch-tower,{'proximity-to-landmarks': {'Palm Jumeirah': '...,"25°10 46.8""N 55°15 53.9""E","{'caption': 'Marble Arch Tower, construction u..."


#### Extract data from tables column and place into new columns

In [1044]:
# For each table with data, extract the data into a dictionary which we will use to create dataframes
# each dataframe contains all table info for each row, which we will concatenate to the global dataframe
companies_associated = {}
key_information = {}
milestones = {}
last_photo = {}
for index, row in renamed_complete_df.iterrows():
    companies_associated[index] = row.tables.get('companies-associated', np.nan)
    key_information[index] = row.tables.get('key-information', np.nan)
    milestones[index] = row.tables.get('milestones', np.nan)
    last_photo[index] = row.last_photo_update

In [1045]:
# build dataframes from the dictionaries created
companies_associated_df = pd.DataFrame(companies_associated).T
key_information_df = pd.DataFrame(key_information).T
milestones_df = pd.DataFrame(milestones).T
last_photo_df = pd.DataFrame(last_photo).T

In [1046]:
# cluster all dataframes into a list for easy access
compound_df_list = [companies_associated_df, key_information_df, milestones_df, last_photo_df]

In [1047]:
# lower case column name and use underscore as space separator
def lower_dash_column_rename(column_list):
    # transform to lower case and use undesrcore as word separator
    return [col.strip().lower().replace(' ', '_') for col in column_list]

# reaname columns
for df in compound_df_list:
    df.columns = lower_dash_column_rename(df.columns)

In [1048]:
# since key_information_df has an area column which clashes in name with the original dataframe /
# we rename this to area_propsearch
key_information_df_column_rename = {'area': 'area_propsearch'}
key_information_df.rename(columns=key_information_df_column_rename, inplace=True)

In [1049]:
# shorted the info caption for the last photo taken of the project, to remove the developer name and keep the date
import re

def shorten_photo_caption(caption):
    # input: caption from caption row
    # output: trimmed caption without the tower name
    # e.g.: Waterfall Tower, construction update May 2021. --> update May 2021
    short_caption = re.search('construction (.*)', caption)
    # check regex found a match, else return original caption
    if short_caption != None:
        return short_caption.group(1).replace('\"', '').replace('.', '')
    return caption
 
# apply shorten_photo_caption on the last_photo_df caption column
last_photo_df.caption = last_photo_df.caption.apply(shorten_photo_caption);


In [1056]:
# concat all table dfs 
compound_info_df = pd.concat(compound_df_list, axis=1)

In [1058]:
# concat concat table df with main df
flat_df_bbay_udev = pd.concat([complete_df, compound_info_df], axis=1)

In [1060]:
# drop 'tables' and 'last_photo_update' columns since these have been flattened into new columns
# before, they used to be compressed into dictionaries in these columns
flat_df_bbay_udev.drop(columns=['tables', 'last_photo_update'], inplace=True)

In [1063]:
flat_df_bbay_udev

,area,sub_area,project,status,url,coordinates,developer,architectural_consultant,contractor,piling_contractor,...,total_units,unit_types,floors_below_ground,height,date_launched,estimated_handover_date,construction_started,revised_estimated_handover_date,caption,photo_url
96,Business Bay,Business Bay,Millennium Binghatti Residences,Under development,https://propsearch.ae/dubai/millennium-binghat...,"25°10 51""N 55°16 8.8""E",Binghatti Developers,Eng. Adnan Saffarini,Granada Europe Engineering Contracting Co.,NaN,...,230,113 studios58 one-bedroom apartments59 two-bed...,NaN,NaN,March 2018,Q4 2019,2018,NaN,update May 2021,https://static.propsearch.ae/dubai-locations/m...
98,Business Bay,Business Bay,Moon Tower,Under development,https://propsearch.ae/dubai/moon-tower,"25°11 3.5""N 55°17 5.1""E",Arabia Group Investment Limited,Chawla Architectural & Consulting Engineers,NaN,NaN,...,255,StudiosOne-bedroom apartmentsTwo-bedroom apart...,NaN,NaN,2007,NaN,2009,Q4 2016,update May 2021,https://static.propsearch.ae/dubai-locations/m...
99,Business Bay,Business Bay,Mövenpick Hotel & Living,Under development,https://propsearch.ae/dubai/movenpick-hotel-li...,"25°11 19.6""N 55°16 13.1""E",SAAS Properties,National Engineering Bureau,Kele Contracting,NaN,...,403,NaN,NaN,NaN,NaN,Q4 2016,2015,NaN,update May 2021,https://static.propsearch.ae/dubai-locations/m...
108,Business Bay,Business Bay,Nobles Residential Tower,Under development,https://propsearch.ae/dubai/nobles-residential...,"25°11 4.5""N 55°17 24.8""E",Tiger Properties,LACASA Architects & Engineering Consultants,Tiger International General Contracting,NaN,...,549,278 one-bedroom apartments239 two-bedroom apar...,2,NaN,September 2017,NaN,2019,NaN,update May 2021,https://static.propsearch.ae/dubai-locations/n...
110,Business Bay,Business Bay,Marble Arch Tower,Under development,https://propsearch.ae/dubai/marble-arch-tower,"25°10 46.8""N 55°15 53.9""E",NaN,DXB Design Studio,Modern Executive Systems Contracting,Geo Foundation Group,...,NaN,NaN,NaN,NaN,NaN,NaN,2019,NaN,update May 2021,https://static.propsearch.ae/dubai-locations/m...
118,Business Bay,Business Bay,Mama Shelter Dubai,Under development,https://propsearch.ae/dubai/mama-shelter-dubai,"25°10 48.3""N 55°16 18.9""E",Kappa Acca Real Estate Development,Franklin Azzi Architecture,CRC Engineering Company LLC,TMF Euro Foundations,...,201,NaN,3,NaN,May 2017,2020,2018,NaN,update May 2021,https://static.propsearch.ae/dubai-locations/m...
121,Business Bay,Business Bay,Lillian Tower,Under development,https://propsearch.ae/dubai/lillian-tower,"25°11 14.3""N 55°15 45.4""E",Marya Investments,NaN,NaN,NaN,...,583,NaN,3,NaN,2007,2016,NaN,NaN,update May 2021,https://static.propsearch.ae/dubai-locations/l...
133,Business Bay,Business Bay,AG Tower Business Bay,Under development,https://propsearch.ae/dubai/ag-tower-business-bay,"25°10 47.6""N 55°16 8.9""E",Arabian Gulf Properties,CVTEC Consulting Engineers,Beijing Emirates International Construction Co.,NaN,...,437,One-bedroom apartmentsTwo-bedroom apartmentsTh...,NaN,NaN,2015,NaN,2016,NaN,update May 2021,https://static.propsearch.ae/dubai-locations/a...
134,Business Bay,Business Bay,Ahad Residences,Under development,https://propsearch.ae/dubai/ahad-residences,"25°11 12""N 55°15 57.6""E",Mercury Investment Holding,VX Architects,Evan Lim Penta Construction LLC,NaN,...,NaN,NaN,5,NaN,2017,NaN,2018,NaN,update May 2021,https://static.propsearch.ae/dubai-locations/a...
140,Business Bay,Business Bay,Paramount Tower Hotel & Residences,Under development,https://propsearch.ae/dubai/paramount-tower-ho...,"25°11 36.3""N 55°15 55.3""E",Damac Properties,LACASA Architects & Engineering Consultants,NaN,NaN,...,826,NaN,NaN,NaN,September 2014,Q3 2019,2015,NaN,update May 2021,https://static.propsearch.ae/dubai-locations/p...


#### Save progress to cvs: Business Bay Under Development (flattened df)

Save progess into csv.  This Df is actionable and ready to be used.  All it needs is contact details.

In [1065]:
flat_df_bbay_udev.to_csv('bbay_udev_flat.csv')

## Block code into functions to run for all areas

### Define main

In [1141]:
# main function
def process_scrap_output(project_status, project_area, df):
    
    # get df window by status and area
    df = df[df.status == project_status]
    df = df[df.area == project_area]
    
    # scrap links in df and build a dictionary with all info
    print('scrapping links')
    scrapped_dict = scrap_df_to_dict(df)
    
    # convert the scrapped dict into a dataframe and concat it with main df
    print('merging scrap with main')
    merged_df = merge_scrapped_dict_to_df(df, scrapped_dict)
    
    # apply rename_table_keys function to rename keys in tables column for easy referencing
    # this removes the project name from the key strings in each dictionary
    print('renaming table keys')
    merged_df.tables.apply(rename_table_keys)
    
    # flatten the info contained in merged_df in dictionaries, into columns on a new df
    print('flattening dfs')
    compound_df = flatten_data_df(merged_df)

    # concat concat table df with main df
    flat_df = pd.concat([merged_df, compound_df], axis=1)

    # drop 'tables' and 'last_photo_update' columns since these have been flattened into new columns
    # before, they used to be compressed into dictionaries in these columns
    flat_df.drop(columns=['tables', 'last_photo_update'], inplace=True)
    
    print('returning flat df')
    
    return flat_df

### Auxiliary functions used in main

In [1158]:
import re

def get_last_photo_update(soup):
    # get string and link of last project photo update to check progress
    construction_updates = soup.find(id='jump-to-construction-updates')
    gallery = construction_updates.find('div', class_='fotorama ps-gallery-portrait')
    raw_caption = gallery.find('a')

    raw_caption = str(raw_caption)
    caption = re.search('data-caption="(.*) Image &amp;copy', raw_caption)
    caption = caption.group(1).replace('\"', '')
    photo_url = re.search('data-full=(.*) href=', raw_caption)
    photo_url = photo_url.group(1).replace('\"', '')
    return caption, photo_url

def scrap_df_to_dict(scrap_df):
    # input: a dataframe with links to scrap from propsearch
    # output: dictionary with scraped data
    # set bounds for urls to scrap
    url_idx_start = 0
    url_idx_end = len(scrap_df) + 1
    # construct an iterable with the original index and the url
    url_list = zip(list(scrap_df.link.iloc[url_idx_start:url_idx_end].index), 
                   list(scrap_df.link.iloc[url_idx_start:url_idx_end]))
    # instantiate dic to hold scrapped data
    scrapped_dict = {}
    # iterate over a list of urls
    for url_key, url in url_list:
        # cast url_key as str to index the dictionary
        url_key = str(url_key)
        # set request header with appropriate permissions
        headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
        # request url data
        r = requests.get(url, headers=headers)#, proxies=proxies)
        # get content
        content = r.content
        # parse content
        soup = BeautifulSoup(content, "html.parser")

        # add this entry to dictionary, using the same key as in the original dataframe index
        scrapped_dict[url_key] = {}
        # save the url
        scrapped_dict[url_key]['url'] = url
        # add space to save each table with info
        scrapped_dict[url_key]['tables'] = {}

        # get coordinates
        coordinates_div = soup.find('div', attrs={'class':'ps-guide-sub-subheading'})
        coordinates = coordinates_div.text.strip().replace('place', '').replace('\'', ' ')
        # add coordinates to dict
        scrapped_dict[url_key]['coordinates'] = coordinates

        # get las photo update
        try:
            caption, photo_url = get_last_photo_update(soup)
            scrapped_dict[url_key]['last_photo_update'] = {'caption': caption, 'photo_url': photo_url}
        except:
            print('Failed to get photo update for: {}'.format(url))
            pass

        # get tables with contents
        kv_list = soup.findAll('div', class_='ps-kv-list')
        # iterate through tables
        for k in kv_list:
            # get table title
            # some tables won't have a title, which we ignore
            try:
                title = k.find(class_='ps-kv-list-title').text.lower().replace(' ', '-')
            except AttributeError as a_err:
                print('Error on table for: {}'.format(url))
                continue
            # instantiate dict for each title
            scrapped_dict[url_key]['tables'][title] = {}
            # get table content
            contents = k.findAll('div', class_='grid-x')
            # iterate through table content
            for content in contents:
                entry = content.text.strip().split('\n')
                scrapped_dict[url_key]['tables'][title][entry[0]] = entry[1]
                
    return scrapped_dict

def merge_scrapped_dict_to_df(df, scrapped_dict):
    # input: df: main df with links and project area details
    # input: scrapped_dict: dict with granular details per project
    # output: merged_df: concat main df with df created with scrapped_dict
    
    # create df from scrapped dictionary, transpose to get the numbers as index
    scrapped_df = pd.DataFrame(scrapped_dict).T
    # scrapped_dict dataframe will have an 'object' type index, whereas the df is 'int64'
    # recast scrapped_dict index to match df, otherwise the concatenation operation returns nan values
    scrapped_df.index = scrapped_df.index.astype('int64')
    # merge with df
    merged_df = pd.concat([df, scrapped_df], axis=1)
    # drop duplicate url-link column
    merged_df.drop(columns=['link'], inplace=True)
    return merged_df
    
    
# function that renames the keys in the tables to remove the project name
def rename_table_keys(table):
    # Input: table: contents of each rown on the tables column of the dataframe
    # eg transform: 'mama-shelter-dubai-key-information' -->  #'key-information'
    # extract all available keys
    key_ids = list(table)
    # iterate over keys and rename them to remove the project name from each and make them universal keys
    for key_id in key_ids:
        if 'key-information' in key_id:
            table['key-information'] = table.pop(key_id)
        elif 'companies-associated' in key_id:
            table['companies-associated'] = table.pop(key_id)
        elif  'milestones' in key_id:
            table['milestones'] = table.pop(key_id)
    return table

# lower case column name and use underscore as space separator
def lower_dash_column_rename(column_list):
    # transform to lower case and use undesrcore as word separator
    return [col.strip().lower().replace(' ', '_') for col in column_list]

# shorted the info caption for the last photo taken of the project, to remove the developer name and keep the date
def shorten_photo_caption(caption):
    # input: caption from caption row
    # output: trimmed caption without the tower name
    # e.g.: Waterfall Tower, construction update May 2021. --> update May 2021
    try:
        short_caption = re.search('construction (.*)', caption)
    except:
        print('error reading caption: {}'.format(caption))
        return caption
    # check regex found a match, else return original caption
    if short_caption != None:
        return short_caption.group(1).replace('\"', '').replace('.', '')
    return caption

def flatten_data_df(df):
    # since scrapped data was saved in dictionaries and placed in cells in the df, we wish to/
    # extract those field and convert them into columns, this function accomplishes that
    # input: df with project data and scrapped data which is saved as dictionaries within the df
    # output: a df that contains all data but as a flat df, no more dictionaries within
    
    # For each table with data, extract the data into a dictionary which we will use to create dataframes
    # each dataframe contains all table info for each row, which we will concatenate to the global dataframe
    companies_associated = {}
    key_information = {}
    milestones = {}
    last_photo = {}
    for index, row in df.iterrows():
        companies_associated[index] = row.tables.get('companies-associated', np.nan)
        key_information[index] = row.tables.get('key-information', np.nan)
        milestones[index] = row.tables.get('milestones', np.nan)
        last_photo[index] = row.last_photo_update

    # build dataframes from the dictionaries created
    companies_associated_df = pd.DataFrame(companies_associated).T
    key_information_df = pd.DataFrame(key_information).T
    try:
        milestones_df = pd.DataFrame(milestones).T
    except ValueError:
        print('issue creating milestones_df: {}'.format(ValueError))
        milestones_df = pd.DataFrame()
    last_photo_df = pd.DataFrame(last_photo).T

    # cluster all dataframes into a list for easy access
    compound_df_list = [companies_associated_df, key_information_df, milestones_df, last_photo_df]

    # reaname columns
    for comp_df in compound_df_list:
        comp_df.columns = lower_dash_column_rename(comp_df.columns)

    # since key_information_df has an area column which clashes in name with the original dataframe /
    # we rename this to area_propsearch
    key_information_df_column_rename = {'area': 'area_propsearch'}
    key_information_df.rename(columns=key_information_df_column_rename, inplace=True)

    # apply shorten_photo_caption on the last_photo_df caption column
    last_photo_df.caption = last_photo_df.caption.apply(shorten_photo_caption);

    # concat all table dfs 
    compound_info_df = pd.concat(compound_df_list, axis=1)

    return compound_info_df


### Define search areas and load data

Load data with links to scrap and specify project status and project area

Requires user input

In [1122]:
# set source file name
source_file_path = 'towers_df.csv'

# set target areas and project status
project_area = 'Business Bay'
project_status = 'Under development'

### DO not change variables below this line
# set output file name
output_file_path = project_area.lower().replace(' ', '_') + '_' + project_status.lower().replace(' ', '_') + '.csv'

# read source file
df = pd.read_csv(source_file_path)

### Run main processing
Outputs csv ready to be used

In [1124]:
flat_df = process_scrap_output(project_status, project_area, df)

Error on table for: https://propsearch.ae/dubai/mama-shelter-dubai
Error on table for: https://propsearch.ae/dubai/amna-tower
Error on table for: https://propsearch.ae/dubai/dorchester-hotel-dubai
Error on table for: https://propsearch.ae/dubai/zada-tower


### Save output to csv

In [1131]:
flat_df.to_csv(os.path.join('output', output_file_path), index=False)

## Create output files for multiple areas

In [1159]:


# set target areas and project status
project_area = 'Motor City'
project_status = 'Under development'

### DO not change variables below this line
# set output file name
output_file_path = project_area.lower().replace(' ', '_') + '_' + project_status.lower().replace(' ', '_') + '.csv'

# set source file name
source_file_path = 'towers_df.csv'
# read source file
df = pd.read_csv(source_file_path)

# run main
flat_df = process_scrap_output(project_status, project_area, df)

# save output
flat_df.to_csv(os.path.join('output', output_file_path), index=False)

scrapping links
merging scrap with main
renaming table keys
flattening dfs
issue creating milestones_df: <class 'ValueError'>
returning flat df


### Describe data to see what to extract

In [1132]:
df = pd.read_csv(source_file_path)

In [1135]:
# project count by status type
df.status.value_counts()

Complete                             934
Under development                    277
Under development (Cancelled)        136
Under development (On hold)          134
Planned                               47
Envisioned                            22
Under development (In progress)       15
Planned (Cancelled)                    5
Under development (Progress slow)      3
Demolished                             2
Complete (Handover underway)           1
Name: status, dtype: int64

In [1136]:
ud_df = df[df.status == 'Under development']

In [1138]:
ud_df.area.value_counts()

Jumeirah Village Circle      97
Business Bay                 34
Arjan                        32
Jumeirah Village Triangle    22
Downtown                     18
Dubai Creek Harbour          17
Palm Jumeirah                16
Sobha Hartland               11
Dubai Hills Estate           10
Dubai Marina                  9
Sports City                   7
Jumeirah Lakes Towers         2
Motor City                    2
Name: area, dtype: int64

Busines Bay
Jumeirah Village Circle
Downtown

Jumeirah Village Circle      97 done
Business Bay                 34 done
Arjan                        32 done
Jumeirah Village Triangle    22 done
Downtown                     18 done
Dubai Creek Harbour          17 done
Palm Jumeirah                16 done
Sobha Hartland               11 done
Dubai Hills Estate           10 done
Dubai Marina                  9 done
Sports City                   7 done
Jumeirah Lakes Towers         2 done
Motor City 

AttributeError: 'DataFrame' object has no attribute 'name'